# SurfaceNets in python Demo

First we load some dependencies. Most probably not all are needed, but we get them anyhow ...

In [24]:
import numpy as np

from collections import defaultdict
from importlib import reload
import ipyvolume as ipv
from stl import mesh
from pythreejs import *


import SurfaceNet

Now we import this python project

In [25]:
SurfaceNet = reload(SurfaceNet)

sn = SurfaceNet.SurfaceNets()

## Example: Volume 2 STL

We now create volume consisting of some ball, boxes, etc.

In [26]:
dim = [0] * 3

dim[0]=100;
dim[1]=100;
dim[2]=100;

vol = np.zeros(dim)

for i in range(0, dim[0]):
    for j in range(0, dim[1]):
        for k in range(0, dim[2]):
            
            # ball
            if(pow(i-dim[0]/2,2)+pow(j-dim[1]/2,2)+pow(k-dim[2]/2,2)<15*15):
                vol[i,j,k]=100
            
            
            # boxes
            if 10 < i < 30 and 10 < j < 30 and 10 < k < 30:
                vol[i,j,k] = 100
                
                
            if 70 < i < 90 and 70 < j < 90 and 10 < k < 50:
                vol[i,j,k] = 100
                
            if 60 < i < 90 and 60 < j < 90 and 10 < k < 30:
                vol[i,j,k] = 100
                
            if 50 < i < 80 and 50 < j < 80 and 50 < k < 80:
                vol[i,j,k] = 100

Let's have a look at this geometry:

In [27]:
ipv.quickvolshow(vol, level=[0.25, 0.75], opacity=0.03, level_width=0.1, data_min=0, data_max=1)

We now apply the surface net algorithm to this volume with a solid-threshold of 50 ( all values > 50 are assumed to be solid).

In [28]:
vertices,faces = sn.surface_net(vol, 50)

print(len(vertices), len(faces))

(100, 100, 100)
[99, 99, 0]
[99, 99, 10]
[99, 99, 20]
[99, 99, 30]
[99, 99, 40]
[99, 99, 50]
[99, 99, 60]
[99, 99, 70]
[99, 99, 80]
[99, 99, 90]
visIdx 970299 1000000
15846 31680


Now that we got vertices and faces, we want to write these into an STL file:

In [29]:
cube = mesh.Mesh(np.zeros(len(faces), dtype=mesh.Mesh.dtype))
for i, f in enumerate(faces):
    
    for j in range(3):
        
        cube.vectors[i][j] = vertices[f[j]]

posElem = defaultdict(list)

for i in range(3):
    for vert in cube.vectors:
        posElem[i].append(vert[i])

#this prints the min/max value per dimension
for posidx in posElem:
    print(posidx, np.min(posElem[posidx]), np.max(posElem[posidx]))
    
cube.save('cube.stl')

0 10.5 89.5
1 10.5 89.5
2 10.5 89.5


Now let's have a look at our 3D model:

In [30]:
# Load the STL files and add the vectors to the plot
your_mesh = mesh.Mesh.from_file('cube.stl')

vertices = BufferAttribute(array=your_mesh.vectors, normalized=True)

geometry = BufferGeometry( attributes={'position': vertices}, )
geometry 

Preview(child=BufferGeometry(attributes={'position': <BufferAttribute shape=(31680, 3, 3), dtype=float32>}), s…